Here, making classification model for the Biclassed dataset. 


In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import os


In [2]:
path = os.path.join(os.getcwd(), )
bank_data = pd.read_csv('bank-full.csv')

FileNotFoundError: File b'bank-full.csv' does not exist

In [ ]:
bank_data.head()

In [ ]:
x = bank_data[bank_data.columns[:-1]]
y = bank_data[bank_data.columns[-1]]

In [ ]:
x.head()

In [ ]:
#sns.pairplot(bank_data, hue = 'y')

In [ ]:
y.head()

Here, i wrote a function which will split data into two seperate dataframes. One is training dataframe and testing dataframe. On the basis of test split percentage.

In [ ]:
def train_test_split(x, y, test_split = 0.5, rand_state = 10):
    mask = np.random.rand(x.shape[0]) < test_split
    x_test = x[mask]
    y_test = y[mask]
    x_train = x[~mask]
    y_train = y[~mask]
    return x_train, x_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 0.2)

In [ ]:
def train_test_split(x, test_split = 0.5, rand_state = 10):
    mask = np.random.rand(x.shape[0]) < test_split
    x_test = x[mask]
    x_train = x[~mask]
    return x_train, x_test

In [ ]:
x_train, x_test = train_test_split(bank_data, 0.2)

In [ ]:
for column in x.columns.values:
    print ("Column {}".format(column))
    print (x[column].unique())

Naive Bayes Classification is a classification algorithm. Where we are using probability of multiple features to predict the class of any probability. 
Naive bayes classifier is based on Naive Bayes Theorem. That theorem is a conditional probability which is gives the probability of any value, Where one even is already happen.

Posterior probability is the probability which comes after performing the operation or we can say posterior probability is the resultant probability. 

Prior probability is the intial probability for a every particular values. 

So,  Formula is:
$$P(A|B) = \frac{P(B|A) P(A)}{P(B)}$$
Where, <br>
$P(A|B)$ is the posterior probability. <br>
$P(A)$ is the prior probability of the class A. <br>
$P(B)$ is the prior probability of the class B. <br>


In our case, we have multiple features or we can say we have multiple values of B.
So, in this case, we always assume that all the features are independent.<br>
The formula is going to modify as.

$$P(class = Y| features...) = \frac{P(features ...| class = Y) P(class = Y)}{P(features...|class = Y)P(class = Y) + P(features... | class N) P(class = N)}$$
<br><br>
$$P\big(class = Y| age = 40\cap job = admin\cap matiral = married \cap education = secondary \cap default = yes \cap balance = 2000 \cap day = 4\cap month = jun\cap duration = 261\cap campaign = 4 \cap pday = 151\cap previous = 5\cap poutcome = other\big) = \frac{P\big(age = 40\cap job = admin\cap matiral = married \cap education = secondary \cap default = yes \cap balance = 2000 \cap day = 4\cap month = jun\cap duration = 261\cap campaign = 4 \cap pday = 151\cap previous = 5\cap poutcome = other | class = Y\big) P(class = Y)}{P\big(age = 40\cap job = admin\cap matiral = married \cap education = secondary \cap default = yes \cap balance = 2000 \cap day = 4\cap month = jun\cap duration = 261\cap campaign = 4 \cap pday = 151\cap previous = 5\cap poutcome = other | class = Y\big) P(class = Y) + P\big(age = 40\cap job = admin\cap matiral = married \cap education = secondary \cap default = yes \cap balance = 2000 \cap day = 4\cap month = jun\cap duration = 261\cap campaign = 4 \cap pday = 151\cap previous = 5\cap poutcome = other | class = N\big) P(class = N)}$$

taking some features 

$$P(class = Y | age = 40\cap job = admin) = \frac{P(age = 40 | class = Y) P(job = admin|class = Y) P(class = Y)}{P(age = 40 | class = Y) P(job = admin|class = Y) P(class = Y) + P(age = 40 | class = N) P(job = admin|class = N) P(class = N)}$$

In [ ]:
class NaiveBayesClassifier (object):
    
    def __init__(self, data, target):
        
        self.data = data
        self.target = target
        self.weights = {}
        
    def estimate_mean_std(self, a, total_sample_size = 1000, each_sample_size = 10):
        
        '''
        Estimating natural paramter of normal distribution
        '''
        
        ele_list = np.array(a)
        random_sample = np.random.choice(ele_list, size = (total_sample_size, each_sample_size))
        
        ## Estimating the value of Mean
        estimate_mean = np.mean(random_sample, axis = 1)
        best_estimate_mean = np.mean(estimate_mean)
        
        ## Taking Best estimate of standard deviation
        estimate_std = np.std(random_sample, axis = 1)
        best_estimate_std = np.mean(estimate_std)
        
        return {'mu':best_estimate_mean, 'sigma' :best_estimate_std}
    
    def estimate_bino_pValue (self, a, total_sample_size = 1000, each_sample_size = 10):
        
        '''
        Estimating natural parameter of binomial distribution
        '''
        uniques = np.unique(a)
        random_sample = np.random.choice(np.array(a), size = (total_sample_size, each_sample_size))
        prob_dict = {}
        for uni in uniques:
            val = []
            for sample in random_sample:
                val.append(np.count_nonzero(uni == sample) / each_sample_size)
            prob_dict[uni] = np.mean(val)
        return prob_dict
    
    def normal_pdf(self, x, mu, sigma):
        
        '''
        Proabability distribution function for Normal Distribution
        '''
        scale_parm = 1 / (np.sqrt(2 * np.pi) * sigma)
        shift = np.e ** -(((x - mu) ** 2) / (2 * sigma ** 2))
        return scale_parm * shift
    
    def calculate_class_prob(self):
        '''
        Calculating class probability for every class
        '''
        self.unique_classes = np.unique(self.data[self.target])
        self.class_prob = {}
        for class_val in self.unique_classes:
            self.class_prob[class_val] = np.count_nonzero(self.data[self.target] == class_val) / self.data.shape[0]

    def calculate_class_based_feature_prob(self, feature):
        
        '''
        Calculating probability for particular feature on the basis of resultant classes
        '''
        val = {}
        for classes in self.unique_classes:
            mask = self.data[self.target] == classes
            feature_now = feature[mask]
            val[classes] = self.estimate_bino_pValue(feature_now)
        return val
    
    def calculate_class_based_normal_feature(self, feature):
        val = {}
        for classes in self.unique_classes:
            mask = self.data[self.target] == classes
            feature_now = feature[mask]
            val[classes] = self.estimate_mean_std(feature_now)
        return val
    
    
    def predict(self, feature):
        
        pred = {}
        pred_class_sum = 0
        pred_class = None
        total = 0
        for classes in self.unique_classes:
            pred[classes] = {}
            sumVal = 1
            # Each feature probability......
            for key, val in feature.items():
                if key in self.weights:
                    
                    if 'mu' in self.weights[key][classes].keys():
                        pred[classes][key] = self.normal_pdf(val, self.weights[key][classes]['mu'], self.weights[key][classes]['sigma'])
                        sumVal *= pred[classes][key]
                    else:
                        pred[classes][key] = self.weights[key][classes][val]
                        sumVal *= pred[classes][key]
            pred[classes]['sum'] = sumVal
            total += sumVal
        
        posterior_prob = {}
        posterior = 0
        classes = 0
        for i in self.unique_classes:
            posterior_prob[i] = pred[i]['sum'] * self.class_prob[i] / total
            if (posterior_prob[i] > posterior):
                posterior = posterior_prob[i]
                classes = i
        return classes
    
    
    def fit(self):
        
        '''
        Fitting data for every feature
        '''
        self.calculate_class_prob()
        self.weights['age'] = self.calculate_class_based_normal_feature(self.data['age'])
        self.weights['job'] = self.calculate_class_based_feature_prob(self.data['job'])
        self.weights['marital'] = self.calculate_class_based_feature_prob(self.data['marital'])
        self.weights['default'] = self.calculate_class_based_feature_prob(self.data['default'])
        self.weights['balance'] = self.calculate_class_based_normal_feature(self.data['balance'])
        self.weights['housing'] = self.calculate_class_based_feature_prob(self.data['housing'])
        self.weights['loan'] = self.calculate_class_based_feature_prob(self.data['loan'])
        self.weights['contact'] = self.calculate_class_based_feature_prob(self.data['contact'])
        self.weights['day'] = self.calculate_class_based_feature_prob(self.data['day'])
        self.weights['month'] = self.calculate_class_based_feature_prob(self.data['month'])
        self.weights['duration'] = self.calculate_class_based_normal_feature(self.data['duration'])
        self.weights['campaign'] = self.calculate_class_based_normal_feature(self.data['campaign'])
        self.weights['pdays'] = self.calculate_class_based_normal_feature(self.data['pdays'])
        self.weights['previous'] = self.calculate_class_based_normal_feature(self.data['previous'])
        self.weights['poutcome'] = self.calculate_class_based_feature_prob(self.data['poutcome'])
        
    def predict_batch(self, data):
        
        classes = data.apply(self.predict,axis = 1)
        return classes
    

In [ ]:
bank_data.columns

In [ ]:
nb = NaiveBayesClassifier(x_train, 'y')


In [ ]:
nb.calculate_class_prob()
nb.class_prob

In [ ]:
nb.fit()

In [ ]:
nb.weights

In [ ]:
nb.weights['age']

In [ ]:
nb.predict(x_test.iloc[0].to_dict())

In [ ]:
pred_classes = nb.predict_batch(x_test)

In [ ]:
class Score_Matrix(object):
    
    '''
    This class has all score matrix by which we can check efficiency of our algorithm on given dataset.
    '''
    
    def __init__(self, y_actual, y_pred):
        self.y_actual = y_actual
        self.y_pred = np.array(y_pred)
        
    def confusion_matrix(self):
        self.data = pd.crosstab(self.y_actual, self.y_pred, rownames= ['actual'], colnames = ['predicted'], margins = True)
        return self.data
        
    def recall(self):
        '''
        data:
            Dataframe of confusion matrix
        '''
        sumVal = 0
        self.confusion_matrix()
        for i in self.data.columns[:-1]:
            sumVal += (self.data[i][i] / self.data['All'][i])
        return np.round(sumVal / (self.data.shape[0] - 1), 3)
    
    def check_accuracy(self):
        sumVal = 0
        self.confusion_matrix()
        for i in self.data.columns[:-1]:
            sumVal += self.data[i][i]
        return np.round(sumVal / self.data['All']['All'], 3)
    
    def precision(self):
        sumVal = 0
        self.confusion_matrix()
        for i in self.data.columns[:-1]:
            sumVal += (self.data[i][i] / self.data[i]['All'])
        avg = sumVal / (self.data.shape[0]- 1)
        return np.round(avg,3)
    

In [ ]:
sm = Score_Matrix(x_test['y'], pred_classes)ac

In [ ]:
print ('Accuracy on test data: {}'.format(sm.check_accuracy()))

In [ ]:
print ('Recall value on test data: {}'.format(sm.recall()))

In [3]:
sm.confusion_matrix()

NameError: name 'sm' is not defined